## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

#1. Dataset
#2. Independent and dependent Features
#3. Cleaing the data (i) Steaming (ii) Stopwords
#4. Fix a sentence length to fix a input
#5. One Hot Representation
#6. Create a model with
##6.1 Embedding Layer
##6.2 LSTM Neural Network
#7. Spliting a data into train test split and Train a model
#8. Calculate Accuracy

#importing data from keggle

In [3]:
!pip install -q kaggle

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [7]:
! mkdir ~/.kaggle

In [8]:
! cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c 'name-of-competition'

In [11]:
! kaggle competitions download -c fake-news

 99% 46.0M/46.5M [00:03<00:00, 16.7MB/s]
100% 46.5M/46.5M [00:03<00:00, 15.0MB/s]


In [12]:
! mkdir train

In [15]:
! unzip fake-news.zip -d train

Archive:  fake-news.zip
  inflating: train/submit.csv        
  inflating: train/test.csv          
  inflating: train/train.csv         


In [16]:
import pandas as pd

In [17]:
df=pd.read_csv('/content/train/train.csv')

In [18]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
df.shape

(20800, 5)

In [20]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [21]:
###Drop Nan Values
df=df.dropna()


In [22]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [23]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [24]:
## Get the Dependent features
y=df['label']

In [25]:
X.shape

(18285, 4)

In [26]:
import tensorflow as tf

In [27]:
tf.__version__

'2.15.0'

In [28]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [29]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [30]:
messages=X.copy()

In [31]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [32]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [40]:
messages.reset_index(inplace=True)

In [41]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [35]:
import nltk
import re
from nltk.corpus import stopwords

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [43]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [44]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2230, 4461, 3903, 4095, 4685, 1053, 2343, 56, 832, 2870],
 [1391, 4323, 2900, 1493, 2507, 1962, 351],
 [4846, 1460, 1678, 3438],
 [3380, 2309, 2011, 435, 2514, 2880],
 [4686, 2507, 2901, 2257, 307, 1208, 2507, 1949, 4267, 1988],
 [3201,
  4370,
  2445,
  4343,
  2431,
  1964,
  691,
  3287,
  1252,
  1501,
  2414,
  1489,
  149,
  4469,
  351],
 [3505, 1177, 4405, 4172, 1598, 1700, 669, 3907, 2384, 1029, 4811],
 [2930, 380, 4653, 4488, 1111, 4739, 1964, 2930, 2384, 1029, 4811],
 [1702, 3350, 1567, 3074, 1038, 3584, 2644, 4135, 1964, 1534],
 [3580, 4065, 3786, 2211, 4943, 3339, 4626, 1957],
 [4106, 562, 1607, 4292, 929, 3480, 3412, 2825, 2281, 2193, 3228],
 [435, 1711, 4685, 3584, 1964, 1111],
 [2895, 1507, 404, 855, 4431, 1135, 2875, 705, 2943],
 [635, 4811, 4132, 692, 260, 251, 4753, 2384, 1029, 4811],
 [1713, 1164, 1191, 4190, 515, 2384, 1029, 4811],
 [793, 4964, 872, 2798, 4606, 1807, 1041, 576, 2198, 725],
 [2619, 580, 4323],
 [2075, 2940, 4120, 4482, 1964, 1048, 2465, 351],
 [29

In [46]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [47]:
onehot_repr[1]

[1391, 4323, 2900, 1493, 2507, 1962, 351]

### Embedding Representation

In [48]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2230 4461 3903 ...    0    0    0]
 [1391 4323 2900 ...    0    0    0]
 [4846 1460 1678 ...    0    0    0]
 ...
 [2125 2110 4331 ...    0    0    0]
 [3787 1111 1404 ...    0    0    0]
 [1494 4825 3815 ...    0    0    0]]


In [49]:
embedded_docs[1]

array([1391, 4323, 2900, 1493, 2507, 1962,  351,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [50]:
embedded_docs[0]

array([2230, 4461, 3903, 4095, 4685, 1053, 2343,   56,  832, 2870,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [51]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [52]:
len(embedded_docs),y.shape

(18285, (18285,))

In [53]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [54]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [56]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 68ms/step - loss: 0.3195 - accuracy: 0.8470 - val_loss: 0.2004 - val_accuracy: 0.9142
Epoch 2/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1308 - accuracy: 0.9522 - val_loss: 0.2076 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0808 - accuracy: 0.9732 - val_loss: 0.2371 - val_accuracy: 0.9142
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0498 - accuracy: 0.9839 - val_loss: 0.3394 - val_accuracy: 0.9097
Epoch 5/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0384 - accuracy: 0.9887 - val_loss: 0.3096 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0273 - accuracy: 0.9913 - val_loss: 0.3255 - val_accuracy: 0.9153
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0226 - accuracy: 0.9931 - val_loss: 0.3570 - val_accuracy: 0.9090

### Adding Dropout

In [57]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [58]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [59]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
